## Guitar amplifier emulation research

In [ ]:
import numpy as np
import seaborn as sns
import keras
import keras.backend as K
import matplotlib.pyplot as plt
import librosa
import tensorflow as tf
import IPython
import git

from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense, Input, Add, Conv1D

from keras.callbacks import BackupAndRestore, CSVLogger, TensorBoard

from callbacks import CheckpointAndSave
from metrics import error_to_signal
from dataset import RawWaveformLoader, WindowWaveformLoader, DataSplitter
from utils import save_attempt_data

Verify GPU support

In [ ]:
tf.config.list_physical_devices()

Defining constants

In [ ]:
INPUT_PATH = "data/float32/test_in.wav"
OUTPUT_PATH = "data/float32/test_out.wav"

TEST_INPUT_PATH = "data/float32/test_in.wav"
TEST_OUTPUT_PATH = "data/float32/test_out.wav"

WORK_DIR = "models"

## Baseline

$a_n = w * h_n + x, n = [0, k)$

Defining required values

In [ ]:
batch_size = 2 ** 8
input_size = 150 # samples

Creating dataset from audio

In [ ]:
train, valid = DataSplitter(input_path=INPUT_PATH, 
                            output_path=OUTPUT_PATH,
                            batch_size=batch_size,
                            input_size=input_size,
                            preemphasis=True).split(RawWaveformLoader)

demo = RawWaveformLoader(input_path=TEST_INPUT_PATH, 
                         output_path=TEST_OUTPUT_PATH, 
                         batch_size=batch_size,
                         input_size=input_size, 
                         duration=10, 
                         offset=86,
                         preemphasis=False)

In [ ]:
learning_rate = 0.001
hidden_units = 64

In [ ]:
K.clear_session()

input_layer = Input((150, 1))
rnn = LSTM(hidden_units, return_sequences=True)(input_layer)
rnn = Dense(1)(rnn)

add = Add()([input_layer, rnn])

model = Model(inputs=input_layer, outputs=[add])

model.compile(optimizer=Adam(learning_rate), loss='mse', metrics=['mse', error_to_signal, 'mae'])

model.summary()

In [ ]:
model_name = input()

if model_name == "":
  raise ValueError("Enter model name")

In [ ]:
subfolder = f"{WORK_DIR}/{model_name}"
save_attempt_data(subfolder, model)

In [ ]:
callbacks = [
  BackupAndRestore(f'{subfolder}/backup'),
  CheckpointAndSave(f'{subfolder}/mse_checkpoint/', demo, model, batch_size, save_best_only=True, save_weights_only=True, monitor='val_mse', verbose=1),
  CheckpointAndSave(f'{subfolder}/ets_checkpoint/', demo, model, batch_size, save_best_only=True, save_weights_only=True, monitor='val_error_to_signal', verbose=1),
  CheckpointAndSave(f'{subfolder}/mae_checkpoint/', demo, model, batch_size, save_best_only=True, save_weights_only=True, monitor='val_mae', verbose=1),
  TensorBoard(log_dir=f'{subfolder}/logs'),
  CSVLogger(f'{subfolder}/logs.csv', append=True),
]

In [ ]:
epochs = 100

model.fit(train, validation_data=valid, shuffle=True, epochs=epochs, callbacks=callbacks)

## Baseline 2

Creating dataset from audio

$a_n = f(a_{n - 1}, a_{n - 2}, \dots, a_{n - k})$

In [ ]:
train, valid = DataSplitter(input_path=INPUT_PATH, 
                            output_path=OUTPUT_PATH,
                            batch_size=batch_size,
                            input_size=input_size,
                            preemphasis=True).split(WindowWaveformLoader)

demo = WindowWaveformLoader(input_path=TEST_INPUT_PATH, 
                         output_path=TEST_OUTPUT_PATH, 
                         batch_size=batch_size,
                         input_size=input_size, 
                         duration=10, 
                         offset=86,
                         preemphasis=False)

In [ ]:
learning_rate = 0.01
conv1d_strides = 12
conv1d_filters = 16
hidden_units = 36

K.clear_session()

input_layer = Input(150, 1)
rnn = LSTM(hidden_units)(input_layer)
rnn = Dense(1)(rnn)

add = Add([input_layer, rnn])

model = Sequential()
model.add(Conv1D(conv1d_filters, 12, strides=conv1d_strides, activation=None, padding='same', input_shape=(input_size,1)))
model.add(Conv1D(conv1d_filters, 12, strides=conv1d_strides, activation=None, padding='same'))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation=None))

model.compile(optimizer=Adam(learning_rate), loss='mse', metrics=['mse', error_to_signal, 'mae'])

model.summary()

In [ ]:
from utils import save_attempt_data
    
subfolder = f"{WORK_DIR}/{model_name}"

save_attempt_data(subfolder, model)

In [ ]:
model.fit(train, validation_data=valid, shuffle=True, epochs=epochs, callbacks=callbacks)

In [ ]:
pred = model.predict(demo)